In [1]:
import pickle
# from utils import plot_optimal_q
import numpy as np
import pandas as pd


In [2]:
df_list = []
for i in range(1, 15):
    with open(r"../data/shA_v4/sarsa_{}.pickle".format(i), "rb") as output_file:
        df_list.append(pickle.load(output_file))

In [3]:
sample = pd.concat(df_list, axis=0)

In [4]:
sample['current_zone'] = [i[0] for i in sample['state']]
sample['current_time'] = [i[1] for i in sample['state']]

In [5]:
action_counts = sample.groupby(['current_time', 'current_zone', 'action'], as_index=False)['episode'].agg('count')
total_counts = sample.groupby(['current_time', 'current_zone'], as_index=False)['episode'].agg('count')
freq = action_counts.merge(total_counts, on=['current_time', 'current_zone'])
freq['freq'] = freq['episode_x']/freq['episode_y']
freq['state'] = [(z,t) for z,t in zip(freq['current_zone'],freq['current_time'])]

In [6]:
ZONE_NUM = 264
TIME_NUM = 96
empty_q = np.array([0.0]*ZONE_NUM)
emp_policy_q = {}
for t in range(TIME_NUM):
    for z in range(ZONE_NUM):
        emp_policy_q[(z,t)] = empty_q.copy()
for s,a,f in zip(freq['state'], freq['action'], freq['freq']):       
    emp_policy_q[s][a] = f     

In [7]:
with open('../output/emp_policy_Q_shA.pickle', 'wb') as handle:
    pickle.dump(emp_policy_q, handle, protocol=pickle.HIGHEST_PROTOCOL)